# RAG 이전 모델

In [ ]:
import gradio as gr
import requests
import base64
import azure.cognitiveservices.speech as speechsdk
import re

# 🔐 API 설정
endpoint_chat = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
key_chat = ""
key_speech = ""
region_speech = "eastus2"

# 📷 이미지 → base64
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except:
        return None

# 🧼 특수문자 제거
def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text) # [ 여기에 넣어서 제거 ]
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()
    
# 🗣️ 음성 생성
def speak_text(text, speed="0%"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="ko-KR-HyunsuMultilingualNeural">
        <prosody rate="{speed}">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=key_speech, region=region_speech)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

# 🤖 GPT 응답 요청2
def get_api_response(max_chars=400, base64_image=None):
    headers = {"Content-Type": "application/json", "api-key": key_chat}
    user_prompt = f"1.이미지는 대한 내용에 대해서 자세하게 설명해줘. 2.한국사능력검정시험에서 자주 출제되는 문제를 기반하여 종합적으로 분석해서 설명해줘. 3.전체 분량은 약 {max_chars}자에 꼭 맞춰서 간결하고 완결성 있게 작성해줘."

    messages = [
        {"role": "system", "content": "한국사능력검정시험 준비를 위한 정보를 제공합니다. 가능한 자세하고 구체적으로 설명해주세요."},
        {"role": "user", "content": user_prompt}
    ]
    
    if base64_image:
        messages.insert(1, {"role": "user", "content": [{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}]})

    payload = {
        "messages": messages,
        "temperature": 0.3,
        "top_p": 0.95,
        "max_tokens": 4096
    }
    try:
        res = requests.post(endpoint_chat, headers=headers, json=payload)
        res.raise_for_status()
        return res.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"API 오류: {e}"

# 🎯 전체 통합 함수
def chatbot_with_duration(image, speed_label, duration_label):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    duration_map = {"1분": 380, "3분": 800, "5분": 2000, "10분": 5000}

    rate = speed_map[speed_label]
    max_chars = duration_map[duration_label]
    base64_image = image_to_base64(image) if image else None

    raw_answer = get_api_response(max_chars, base64_image)
    cleaned = clean_special_characters(raw_answer)
    audio = speak_text(cleaned, rate)

    return cleaned, audio

# 🖼️ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 요약 및 음성화")
    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="말하기 속도", value="1배속")
            duration = gr.Radio(["1분", "3분", "5분", "10분"], label="TTS 길이", value="1분")
            submit = gr.Button("응답 받기")
        with gr.Column():
            gr.Markdown("### 📜 응답")
            answer = gr.Textbox(label="GPT 응답", lines=10)
            gr.Markdown("### 🗣️들으면서 공부하자")
            audio = gr.Audio(label="음성 출력", autoplay=True)

    image.change(fn=chatbot_with_duration, inputs=[image, speed, duration], outputs=[answer, audio])

demo.launch()

# RAG 모델

In [ ]:
import gradio as gr
import requests
import base64
import azure.cognitiveservices.speech as speechsdk
import re

# 🔐 API 설정
endpoint_chat = "https://6b013-azure-ai-service.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
key_chat = ""
search_endpoint="https://6b013-ai-search-3.search.windows.net"
search_key=""
search_index="history-index"

key_speech = ""
region_speech = "eastus2"

# 📷 이미지 → base64
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except:
        return None

# 🧼 특수문자 제거
def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text) # [ 여기에 넣어서 제거 ]
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()
    
# 🗣️ 음성 생성
def speak_text(text, speed="0%"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="ko-KR-HyunsuMultilingualNeural">
        <prosody rate="{speed}">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=key_speech, region=region_speech)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

# 🤖 GPT 응답 요청
def get_api_response_1(max_chars=400, base64_image=None):
    headers = {"Content-Type": "application/json", "api-key": key_chat}
    user_prompt = f"1.이미지는 대한 내용에 대해서 자세하게 설명해줘. 2.한국사능력검정시험에서 자주 출제되는 문제를 기반하여 종합적으로 분석해서 설명해줘. 3.전체 분량은 약 {max_chars}자에 꼭 맞춰서 간결하고 완결성 있게 작성해줘."

    messages = [
        {"role": "system", "content": "한국사능력검정시험 준비를 위한 정보를 제공합니다. 가능한 자세하고 구체적으로 설명하며, 이모지,*,/,#을 포함하지않는 줄글 형식으로 작성합니다."},
        {"role": "user", "content": user_prompt}
    ]
    if base64_image:
        messages.insert(1, {"role": "user", "content": [{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}]})

    payload = {
  "messages": messages,
  "temperature": 0.3,
  "top_p": 0.95,
  "max_tokens": 4000,
  "data_sources": [
    {
      "type": "azure_search",
      "parameters": {
        "endpoint": "https://6b013-ai-search-3.search.windows.net",
        "index_name": "history-index",
        "semantic_configuration": "history",
        "query_type": "semantic",
        "fields_mapping": {
          "content_fields": ["content"],
          "title_field": "title"
        },
        "in_scope": True,
        "strictness": 3,
        "top_n_documents": 5,
        "authentication": {
          "type": "api_key",
          "key": ""
        }
      }
    }
  ]
}
    try:
        res = requests.post(endpoint_chat, headers=headers, json=payload)
        res.raise_for_status()
        return res.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"API 오류: {e}"

# 🎯 전체 통합 함수
def chatbot_with_duration(image, speed_label, duration_label):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    duration_map = {"1분": 380, "3분": 800, "5분": 2000, "10분": 5000}

    rate = speed_map[speed_label]
    max_chars = duration_map[duration_label]
    base64_image = image_to_base64(image) if image else None

    raw_answer = get_api_response_1(max_chars, base64_image)
    cleaned = clean_special_characters(raw_answer)
    audio = speak_text(cleaned, rate)

    return cleaned, audio

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 요약 및 음성화")
    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="말하기 속도", value="1배속")
            duration = gr.Radio(["1분", "3분", "5분", "10분"], label="TTS 길이", value="1분")
            submit = gr.Button("응답 받기")
        with gr.Column():
            gr.Markdown("### 📜 응답")
            answer = gr.Textbox(label="GPT 응답", lines=10)
            gr.Markdown("### 🗣️ 들으면서 공부하자")
            audio = gr.Audio(label="음성 출력", autoplay=True)

    submit.click(fn=chatbot_with_duration, inputs=[image, speed, duration], outputs=[answer, audio])

demo.launch()

# 챗봇2개 섞어보기

In [ ]:
import gradio as gr
import requests
import base64
import re
import azure.cognitiveservices.speech as speechsdk

# 🔐 API 키 및 엔드포인트 설정
openai_vision_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_vision_key = ""

openai_rag_endpoint = "https://6b013-azure-ai-service.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"
openai_rag_key = ""

search_endpoint = "https://6b013-ai-search-3.search.windows.net"
search_key = ""

speech_key = ""
speech_region = "eastus2"

# 📷 이미지 → base64
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except:
        return None

# 🧼 특수문자 제거
def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text)
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

# 🗣️ 음성 생성
def speak_text(text, speed="0%"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="ko-KR-HyunsuMultilingualNeural">
        <prosody rate="{speed}">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

# 🤖 GPT Vision 요약 - RAG 친화형
def get_vision_summary(base64_image, max_chars=600):
    headers = {"Content-Type": "application/json", "api-key": openai_vision_key}
    messages = [
        {
            "role": "system",
            "content": (
                "이 이미지는 한국사능력검정시험 필기 자료입니다. "
                "이미지의 내용을 분석하여 RAG 모델이 이해할 수 있도록 "
                "완성된 문장과 구조로 정리된 요약 지식을 작성하세요. "
                "불완전한 메모가 아니라, 설명형 텍스트로 작성하고 문장은 자연스럽게 끝나야 합니다."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {
                    "type": "text",
                    "text": (
                        f"이 이미지를 바탕으로 한국사능력검정시험 스타일로 {max_chars}자 이내로 핵심 개념과 맥락을 정리해주세요. "
                        "사건, 시대, 인물, 개념을 중심으로 서술형으로 작성하며, 문장은 완결된 줄글 형태로 마무리하세요."
                    )
                }
            ]
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.3,
        "top_p": 0.95,
        "max_tokens": 4096
    }
    res = requests.post(openai_vision_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# 🤖 GPT RAG 답변 - Vision 지문 기반 고급 응답
def get_rag_answer(input_text, max_chars=800):
    headers = {"Content-Type": "application/json", "api-key": openai_rag_key}
    messages = [
        {
            "role": "system",
            "content": (
                "당신은 한국사능력검정시험 전문가입니다. "
                "사용자가 제공하는 필기 요약 지문을 바탕으로 출제자의 관점에서 "
                "중요한 배경지식, 핵심개념, 역사적 비교나 연관성을 포함한 응답을 작성합니다. "
                "신뢰도 높은 사료와 관련 키워드를 중심으로 설명하세요."
            )
        },
        {
            "role": "user",
            "content": (
                f"다음은 필기 이미지로부터 생성된 요약 지문입니다. "
                f"{max_chars}자 이내로 한국사능력검정시험 스타일로 마무리된 설명문을 작성해주세요.\n\n"
                f"요약 지문:\n\"\"\"\n{input_text}\n\"\"\""
            )
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": 4096,
        "frequency_penalty": 0.25,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": "history-index",
                    "semantic_configuration": "history",
                    "query_type": "semantic",
                    "fields_mapping": {
                        "content_fields": ["content"],
                        "title_field": "title"
                    },
                    "in_scope": True,
                    "strictness": 3,
                    "top_n_documents": 3,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    }
                }
            }
        ]
    }
    res = requests.post(openai_rag_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# 🎯 통합 파이프라인
def process_image_pipeline(image, speed_label, duration_label):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    duration_map = {"1분": 600, "3분": 1500, "5분": 3500, "10분": 5000}

    rate = speed_map[speed_label]
    max_chars = duration_map[duration_label]
    base64_image = image_to_base64(image) if image else None

    vision_summary = get_vision_summary(base64_image, max_chars=max_chars)
    rag_answer = get_rag_answer(vision_summary, max_chars=max_chars)
    cleaned = clean_special_characters(rag_answer)
    audio = speak_text(cleaned, rate)

    return vision_summary, cleaned, audio

# 🖼️ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 → 요약 → RAG 응답 → 음성 출력")
    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="말하기 속도", value="1배속")
            duration = gr.Radio(["1분", "3분", "5분", "10분"], label="TTS 길이", value="1분")
            submit = gr.Button("응답 받기")
        with gr.Column():
            vision_output = gr.Textbox(label="1️⃣ GPT Vision 요약", lines=3)
            rag_output = gr.Textbox(label="2️⃣ RAG 최종 응답", lines=10)
            audio = gr.Audio(label="🎧 음성 출력", autoplay=False)

    submit.click(fn=process_image_pipeline, inputs=[image, speed, duration], outputs=[vision_output, rag_output, audio])

demo.launch()

# 최종

In [ ]:
import gradio as gr
import requests
import base64
import re
import azure.cognitiveservices.speech as speechsdk

# 🔐 API 키 및 엔드포인트 설정
openai_vision_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_vision_key = ""

openai_rag_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_rag_key = ""

search_endpoint = "https://6a026-proj2-stor.search.windows.net"
search_key = ""

speech_key = ""
speech_region = "eastus2"

# 이미지 → base64
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except:
        return None

# 특수문자 제거
def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text)
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

# 음성 생성
def speak_text(text, speed="0%"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="ko-KR-HyunsuMultilingualNeural">
        <prosody rate="{speed}">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

# GPT Vision 요약
def get_vision_summary(base64_image):
    headers = {"Content-Type": "application/json", "api-key": openai_vision_key}
    messages = [
        {
            "role": "system",
            "content": (
                "이 이미지는 한국사능력검정시험 필기 자료입니다. "
                "이미지의 내용을 분석하여 RAG 모델이 이해할 수 있도록 "
                "완성된 문장과 구조로 정리된 요약 지식을 작성하세요."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {
                    "type": "text",
                    "text": (
                        f"이미지의 내용을 한국사능력검정시험 개념과 맥락으로 키워드중심으로 모두 대답해주세요. "
                        "사건, 시대, 인물, 개념을 중심으로 서술형으로 작성하며, 문장은 완결된 줄글 형태로 마무리하세요."
                    )
                }
            ]
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.95,
        "max_tokens": 4096
    }
    res = requests.post(openai_vision_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# GPT RAG 응답
def get_rag_answer_with_citations(input_text, max_chars=350):
    headers = {"Content-Type": "application/json", "api-key": openai_rag_key}
    messages = [
        {
            "role": "system",
            "content": (
                "당신은 한국사능력검정시험 전문가입니다. "
                "사용자가 제공하는 키워드중심 필기 요약 지문을 바탕으로 출제자의 관점에서 한국사능력검정시험에 도움이 되는 내용을 중점으로 응답을 작성하세요"
                "문서를 기반한 내용을 포함하여 신뢰도 높은 사료와 관련 키워드를 중심으로 설명하세요."
            )
        },
        {
            "role": "user",
            "content": (
                f"{input_text}를(을) 문서를 기반으로 {max_chars}자에 근접하게 맞추어 한국사능력검정시험을 응시자에게 도움이되게 설명문을 작성해주세요."
            )
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": 4096,
        "frequency_penalty": 0.25,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": "index-026history-csvfile",
                    "semantic_configuration": "026history-csvfile-semantic",
                    "query_type": "semantic",
                    "fields_mapping": {
                        "content_fields": ["tags", "content", "category", "title"],
                        "title_field": "title"
                    },
                    "in_scope": True,
                    "role_information": "",
                    "filter": None,
                    "strictness": 3,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    },
                    "key": search_key,
                    "indexName": "index-026history-csvfile"
                }
            }
        ]
    }
    res = requests.post(openai_rag_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    result = res.json()
    content = result["choices"][0]["message"]["content"]
    citations = result["choices"][0]["message"].get("context", {}).get("citations", [])
    return {"answer": content, "citations": citations}

# 🔗 출처 포맷
def format_citations(citations):
    if not citations:
        return "🔍 관련 문서 없음"
    formatted = []
    for i, c in enumerate(citations, 1):
        title = c.get("title", "제목 없음")
        url = c.get("url", "")
        content_citation = c.get("content", "")
        entry = f"[{i}] {title} - {content_citation}"
        # URL이 있는 경우에만 추가
        if url:
            entry += f" - {url}"
        formatted.append(entry)
    return "\n".join(formatted)


# 🧠 전체 파이프라인 함수
def process_image_pipeline(image, speed_label, duration_label):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    duration_map = {"1분": 300, "3분": 700, "5분": 3000}

    rate = speed_map[speed_label]
    max_chars = duration_map[duration_label]
    base64_image = image_to_base64(image) if image else None

    vision_summary = get_vision_summary(base64_image)
    rag_result = get_rag_answer_with_citations(vision_summary, max_chars=max_chars)

    rag_text = clean_special_characters(rag_result["answer"])
    citations = format_citations(rag_result.get("citations", []))
    audio = speak_text(rag_text, rate)

    return vision_summary, rag_text, audio, citations

# 🧩 Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 → GPT 요약 → RAG 응답 → 음성 출력 + 🔗 출처 문서 표시")

    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="말하기 속도", value="1배속")
            duration = gr.Radio(["1분", "3분", "5분"], label="TTS 길이", value="1분")
            submit = gr.Button("응답 받기")
        with gr.Column():
            vision_output = gr.Textbox(label="1️⃣ GPT Vision 요약", lines=3)
            rag_output = gr.Textbox(label="2️⃣ RAG 최종 응답", lines=10)
            audio = gr.Audio(label="🎧 음성 출력", autoplay=False)
            citations_box = gr.Textbox(label="🔗 출처 문서", lines=5)

    submit.click(fn=process_image_pipeline,
                 inputs=[image, speed, duration],
                 outputs=[vision_output, rag_output, audio, citations_box])

demo.launch()

# 도식화하기
---

In [ ]:
import gradio as gr
import requests
import base64
import re
from graphviz import Digraph
import azure.cognitiveservices.speech as speechsdk

# 환경변수에서 API 키 등 로드 (또는 수동 입력 가능)
openai_vision_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_vision_key = ""

openai_rag_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_rag_key = ""

search_endpoint = "https://6a026-proj2-stor.search.windows.net"
search_key = ""

speech_key = ""
speech_region = "eastus2"

# === 유틸리티 함수들 ===
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except:
        return None

def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text)
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

def speak_text(text, speed="0%"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="ko-KR-HyunsuMultilingualNeural">
        <prosody rate="{speed}">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

def get_vision_summary(base64_image):
    headers = {"Content-Type": "application/json", "api-key": openai_vision_key}
    messages = [
        {
            "role": "system",
            "content": (
                "이 이미지는 한국사능력검정시험 필기 자료입니다. "
                "이미지의 내용을 분석하여 RAG 모델이 이해할 수 있도록 "
                "완성된 문장과 구조로 정리된 요약 지식을 작성하세요."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {
                    "type": "text",
                    "text": (
                        f"이미지의 내용을 한국사능력검정시험 개념과 맥락으로 키워드중심으로 모두 대답해주세요. "
                        "사건, 시대, 인물, 개념을 중심으로 서술형으로 작성하며, 문장은 완결된 줄글 형태로 마무리하세요."
                    )
                }
            ]
        }
    ]
    payload = {"messages": messages, "temperature": 0.2, "top_p": 0.95, "max_tokens": 4096}
    res = requests.post(openai_vision_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

def get_rag_answer_with_citations(input_text, max_chars=350):
    headers = {"Content-Type": "application/json", "api-key": openai_rag_key}
    messages = [
        {
            "role": "system",
            "content": (
                "당신은 한국사능력검정시험 전문가입니다. "
                "사용자가 제공하는 키워드중심 필기 요약 지문을 바탕으로 출제자의 관점에서 한국사능력검정시험에 도움이 되는 내용을 중점으로 응답을 작성하세요"
                "문서를 기반한 내용을 포함하여 신뢰도 높은 사료와 관련 키워드를 중심으로 설명하세요."
            )
        },
        {
            "role": "user",
            "content": (
                f"{input_text}를(을) 문서를 기반으로 {max_chars}자에 근접하게 맞추어 한국사능력검정시험을 응시자에게 도움이되게 설명문을 작성해주세요."
            )
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": 4096,
        "frequency_penalty": 0.25,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": "index-026history-csvfile",
                    "semantic_configuration": "026history-csvfile-semantic",
                    "query_type": "semantic",
                    "fields_mapping": {
                        "content_fields": ["tags", "content", "category", "title"],
                        "title_field": "title"
                    },
                    "in_scope": True,
                    "strictness": 3,
                    "top_n_documents": 3,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    }
                }
            }
        ]
    }
    res = requests.post(openai_rag_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    result = res.json()
    content = result["choices"][0]["message"]["content"]
    citations = result["choices"][0]["message"].get("context", {}).get("citations", [])
    return {"answer": content, "citations": citations}

# 출처참조 텍스트 정리
def format_citations(citations):
    if not citations:
        return "🔍 관련 문서 없음"
    formatted = []
    for i, c in enumerate(citations, 1):
        title = c.get("title", "제목 없음")
        url = c.get("url", "")
        content_citation = c.get("content", "")
        entry = f"[{i}] {title} - {content_citation}"
        if url:
            entry += f" - {url}"
        formatted.append(entry)
    return "\n".join(formatted)

# 줄바꿈 함수 (한 줄당 최대 40자)
def wrap_text(text, max_len=40):
    return "\\n".join(re.findall(f".{{1,{max_len}}}", text.strip()))

# 트리 시각화 (출처 기반)
def generate_citation_tree(text, citations, max_nodes_per_citation=3):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR', splines='true', dpi='300')
    dot.attr(ratio='auto', nodesep='0.7', ranksep='1', margin='0.3,0.3')
    dot.attr('node', fontname="Malgun Gothic", fontsize='10', margin='0.1,0.05',
             shape='box', style='rounded,filled', fillcolor='lightgrey')
    dot.node('summary', '📖 요약문장 도식화')

    # 문장을 . ! ? 로 나누기
    sentences = re.split(r'[.!?]\s*', text.strip())

    for i, citation in enumerate(citations, 1):
        title = citation.get("title", f"출처 {i}")
        content = citation.get("content", "")
        src_node = f"src_{i}"

        label = wrap_text(f"[{i}] {title}\n{content[:-len(title)]}")
        dot.node(src_node, label, shape="ellipse", fillcolor="lightyellow")
        dot.edge("summary", src_node)

        child_count = 0  # 연결된 문장 수 추적

        for j, sent in enumerate(sentences):
            if child_count >= max_nodes_per_citation:
                break  # 최대 개수 초과 시 중단
            if any(kw in sent for kw in [title] + content.split()):
                node_id = f"{src_node}_{j}"
                wrapped_sent = wrap_text(sent, max_len=35)
                dot.node(node_id, wrapped_sent, shape="note", fillcolor="white")
                dot.edge(src_node, node_id)
                child_count += 1  # 문장 연결 수 카운트

    return dot.render("tree_output", format='png', cleanup=False)

# 전체 파이프라인 실행 함수
def process_image_pipeline(image, speed_label, duration_label):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    duration_map = {"1분": 300, "3분": 700, "5분": 2000}

    rate = speed_map[speed_label]
    max_chars = duration_map[duration_label]
    base64_image = image_to_base64(image) if image else None

    vision_summary = get_vision_summary(base64_image)
    rag_result = get_rag_answer_with_citations(vision_summary, max_chars=max_chars)
    rag_text = clean_special_characters(rag_result["answer"])
    citations = rag_result.get("citations", [])
    citations_text = format_citations(citations)
    audio = speak_text(rag_text, rate)
    tree_path = generate_citation_tree(rag_text, citations)

    return vision_summary, rag_text, audio, citations_text, tree_path

# Gradio UI 구성
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 → GPT 요약 → RAG 응답 → 음성 출력 + 출처 기반 트리 도식화")
    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="📷 필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="🗣️ 말하기 속도", value="1배속")
            duration = gr.Radio(["1분", "3분", "5분"], label="⏱️ TTS 길이", value="1분")
            submit = gr.Button("🧠 응답 받기")
        with gr.Column():
            vision_output = gr.Textbox(label="1️⃣ GPT Vision 요약", lines=3)
            rag_output = gr.Textbox(label="2️⃣ RAG 최종 응답", lines=10)
            audio = gr.Audio(label="🎧 음성 출력", autoplay=False)
            citations_box = gr.Textbox(label="🔗 출처 문서", lines=5)
    with gr.Row():
        tree_output = gr.Image(label="🌳 출처 기반 트리 시각화", type="filepath")

    submit.click(fn=process_image_pipeline,
                 inputs=[image, speed, duration],
                 outputs=[vision_output, rag_output, audio, citations_box, tree_output])

demo.launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.


---

In [ ]:
import gradio as gr
import re
import base64
from graphviz import Digraph
import azure.cognitiveservices.speech as speechsdk
import requests

# 🔐 API 키 및 환경 설정
openai_vision_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_vision_key = ""

openai_rag_endpoint = "https://a026-proj2-openai2.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2024-02-15-preview"
openai_rag_key = ""

search_endpoint = "https://6a026-proj2-stor.search.windows.net"
search_key = ""

speech_key = ""
speech_region = "eastus2"

# 📸 이미지 → base64
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except:
        return None

# ✂️ 텍스트 클린업
def clean_special_characters(text):
    text = re.sub("[\U00010000-\U0010ffff]", "", text)
    text = re.sub(r'[*#\\/]', '', text)
    text = re.sub(r'\n', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

# 🔈 음성 생성
def speak_text(text, speed="0%"):
    ssml = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="ko-KR">
      <voice name="ko-KR-HyunsuMultilingualNeural">
        <prosody rate="{speed}">{text}</prosody>
      </voice>
    </speak>
    """
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
    result = synthesizer.speak_ssml_async(ssml).get()
    return result.audio_data if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted else None

# 🧠 Vision 요약
def get_vision_summary(base64_image):
    headers = {"Content-Type": "application/json", "api-key": openai_vision_key}
    messages = [
        {
            "role": "system",
            "content": (
                "이 이미지는 한국사능력검정시험 필기 자료입니다. "
                "이미지의 내용을 분석하여 RAG 모델이 이해할 수 있도록 "
                "완성된 문장과 구조로 정리된 요약 지식을 작성하세요."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {
                    "type": "text",
                    "text": (
                        f"이미지의 내용을 한국사능력검정시험 개념과 맥락으로 키워드중심으로 모두 대답해주세요. "
                        "사건, 시대, 인물, 개념을 중심으로 서술형으로 작성하며, 문장은 완결된 줄글 형태로 마무리하세요."
                    )
                }
            ]
        }
    ]
    payload = {"messages": messages, "temperature": 0.2, "top_p": 0.95, "max_tokens": 4096}
    res = requests.post(openai_vision_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    return res.json()["choices"][0]["message"]["content"]

# 📚 RAG 응답
def get_rag_answer_with_citations(input_text, max_chars=350):
    headers = {"Content-Type": "application/json", "api-key": openai_rag_key}
    messages = [
        {
            "role": "system",
            "content": (
                "당신은 한국사능력검정시험 전문가입니다. "
                "사용자가 제공하는 키워드중심 필기 요약 지문을 바탕으로 출제자의 관점에서 한국사능력검정시험에 도움이 되는 내용을 중점으로 응답을 작성하세요"
                "문서를 기반한 내용을 포함하여 신뢰도 높은 사료와 관련 키워드를 중심으로 설명하세요."
            )
        },
        {
            "role": "user",
            "content": (
                f"{input_text}를(을) 문서를 기반으로 {max_chars}자에 근접하게 맞추어 한국사능력검정시험을 응시자에게 도움이되게 설명문을 작성해주세요."
            )
        }
    ]
    payload = {
        "messages": messages,
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": 4096,
        "frequency_penalty": 0.25,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": "index-026history-csvfile",
                    "semantic_configuration": "026history-csvfile-semantic",
                    "query_type": "semantic",
                    "fields_mapping": {
                        "content_fields": ["tags", "content", "category", "title"],
                        "title_field": "title"
                    },
                    "in_scope": True,
                    "strictness": 3,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    }
                }
            }
        ]
    }
    res = requests.post(openai_rag_endpoint, headers=headers, json=payload)
    res.raise_for_status()
    result = res.json()
    return {
        "answer": result["choices"][0]["message"]["content"],
        "citations": result["choices"][0]["message"].get("context", {}).get("citations", [])
    }

# 🔗 출처 포맷
def format_citations(citations):
    if not citations:
        return "🔍 관련 문서 없음"
    formatted = []
    for i, c in enumerate(citations, 1):
        title = c.get("title", "제목 없음")
        url = c.get("url", "")
        content_citation = c.get("content", "")
        entry = f"[{i}] {title} - {content_citation}"
        if url:
            entry += f" - {url}"
        formatted.append(entry)
    return "\n".join(formatted)

# ✂️ 텍스트 줄바꿈
def wrap_text(text, max_len=40, max_lines=3):
    lines = re.findall(f".{{1,{max_len}}}", text.strip())
    if len(lines) > max_lines:
        return "\\n".join(lines[:max_lines]) + "\\n..."
    return "\\n".join(lines)

# 🌳 출처 기반 트리 생성
def generate_citation_tree(text, citations, max_nodes_per_citation=3):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR', splines='true', dpi='300')
    dot.attr(ratio='auto', nodesep='0.7', ranksep='1', margin='0.3,0.3')
    dot.attr('node', fontname="Malgun Gothic", fontsize='10', margin='0.1,0.05',
             shape='box', style='rounded,filled', fillcolor='lightgrey')
    dot.node('summary', '📖 요약문장 도식화')

    sentences = re.split(r'[.!?]\s*', text.strip())

    for i, citation in enumerate(citations, 1):
        title = citation.get("title", f"출처 {i}")
        content = citation.get("content", "")
        src_node = f"src_{i}"
        label = wrap_text(f"[{i}] {title}\n{content[:-len(title)]}")
        dot.node(src_node, label, shape="ellipse", fillcolor="lightyellow")
        dot.edge("summary", src_node)

        child_count = 0
        for j, sent in enumerate(sentences):
            if child_count >= max_nodes_per_citation:
                break
            if any(kw in sent for kw in [title] + content.split()):
                node_id = f"{src_node}_{j}"
                wrapped_sent = wrap_text(sent, max_len=35)
                dot.node(node_id, wrapped_sent, shape="note", fillcolor="white")
                dot.edge(src_node, node_id)
                child_count += 1

    return dot.render("tree_output", format='png', cleanup=False)

# 🚀 전체 파이프라인
def process_image_pipeline(image, speed_label, duration_label, max_nodes_per_citation):
    speed_map = {"1배속": "0%", "1.5배속": "50%", "2배속": "100%"}
    duration_map = {"1분": 300, "3분": 700, "5분": 2000}
    rate = speed_map[speed_label]
    max_chars = duration_map[duration_label]

    base64_image = image_to_base64(image) if image else None
    vision_summary = get_vision_summary(base64_image)
    rag_result = get_rag_answer_with_citations(vision_summary, max_chars=max_chars)
    rag_text = clean_special_characters(rag_result["answer"])
    citations = rag_result.get("citations", [])
    citations_text = format_citations(citations)
    audio = speak_text(rag_text, rate)
    tree_path = generate_citation_tree(rag_text, citations, max_nodes_per_citation=max_nodes_per_citation)

    return vision_summary, rag_text, audio, citations_text, tree_path

# 🧩 Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 📚 한국사 필기노트 → GPT 요약 → RAG 응답 → 음성 출력 + 출처 기반 트리 도식화")

    with gr.Row():
        with gr.Column():
            image = gr.Image(type="filepath", label="📷 필기 이미지 업로드")
            speed = gr.Radio(["1배속", "1.5배속", "2배속"], label="🗣️ 말하기 속도", value="1배속")
            duration = gr.Radio(["1분", "3분", "5분"], label="⏱️ TTS 길이", value="1분")
            max_nodes_slider = gr.Slider(minimum=1, maximum=10, step=1, value=3,
                                         label="🌿 출처당 연결 문장 수 (노드 제한)")
            submit = gr.Button("🧠 응답 받기")

        with gr.Column():
            vision_output = gr.Textbox(label="1️⃣ GPT Vision 요약", lines=3)
            rag_output = gr.Textbox(label="2️⃣ RAG 최종 응답", lines=10)
            audio = gr.Audio(label="🎧 음성 출력", autoplay=False)
            citations_box = gr.Textbox(label="🔗 출처 문서", lines=5)

    with gr.Row():
        tree_output = gr.Image(label="🌳 출처 기반 트리 시각화", type="filepath")

    submit.click(
        fn=process_image_pipeline,
        inputs=[image, speed, duration, max_nodes_slider],
        outputs=[vision_output, rag_output, audio, citations_box, tree_output]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
